## Ablation study on KS dataset for PFNN contract steps

In [ ]:
cd ..

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from model.utilities import *
from model.vae_base import *
import sys
sys.path.append('./model')
from sklearn.decomposition import PCA

import numpy.random as random
from scipy.stats import gaussian_kde


font = {'size'   : 12, 'family': 'Times New Roman'}
matplotlib.rc('font', **font)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

# Main
n_train = 900
n_test = 100

batch_size = 100

sub = 4 # spatial subsample
S = 512
s = S//sub

T_in = 500 # focus on measure invariant set
T = 1100 # seconds to extract from each trajectory in data
T_out = T_in + T
step = 1 # Seconds to learn solution operator

# Load data
predloader = MatReader('./data/KS.mat') # load the generated data
data_raw = predloader.read_field('u')
data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]

# randomly sample half episodes from the train data episodes to avoid biased test distribution
episode_samples = int(0.5*n_train)
data_sampled_train = data_tensor[torch.randperm(data_tensor[:n_train].size(0))[:episode_samples],:,:]
data_test = data_tensor[-n_test:,:,:]

train_sample = data_sampled_train[:,T_in:T_out,:].reshape(-1, s)
test_a = data_test[:,T_in-1:T_out-1,:].reshape(-1, s)
test_u = data_test[:,T_in:T_out,:].reshape(-1, s)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device('cpu')

PFNN_step_100_path = 'fill_PFNN_C_lsi_1_model_path'
model_step_100 = torch.load(PFNN_step_100_path, map_location=device)

PFNN_step_300_path = 'fill_PFNN_C_lsi_3_model_path'
model_step_300 = torch.load(PFNN_step_300_path, map_location=device)

PFNN_step_500_path = 'fill_PFNN_C_lsi_5_model_path'
model_step_500 = torch.load(PFNN_step_500_path, map_location=device)

PFNN_step_700_path = 'fill_PFNN_C_lsi_7_model_path'
model_step_700 = torch.load(PFNN_step_700_path, map_location=device)

PFNN_step_900_path = 'fill_PFNN_C_lsi_9_model_path'
model_step_900 = torch.load(PFNN_step_900_path, map_location=device)

In [ ]:
reduced_model1_long_pred = []
reduced_model2_long_pred = []
reduced_model3_long_pred = []
reduced_model4_long_pred = []
reduced_model5_long_pred = []

with torch.no_grad():
      for init_id in range(n_test):
            step_100_long_pred = long_prediction(model_step_100, test_a, init_id*T, 1, s, s, T=T)
            step_300_long_pred = long_prediction(model_step_300, test_a, init_id*T, 1, s, s, T=T)
            step_500_long_pred = long_prediction(model_step_500, test_a, init_id*T, 1, s, s, T=T)
            step_700_long_pred = long_prediction(model_step_700, test_a, init_id*T, 1, s, s, T=T)
            step_900_long_pred = long_prediction(model_step_900, test_a, init_id*T, 1, s, s, T=T)

            reduced_model1_long_pred.append(pca.transform(step_100_long_pred[~torch.isnan(step_100_long_pred).any(dim=1) & ~torch.isinf(step_100_long_pred).any(dim=1)]))
            reduced_model2_long_pred.append(pca.transform(step_300_long_pred))
            reduced_model3_long_pred.append(pca.transform(step_500_long_pred))
            reduced_model4_long_pred.append(pca.transform(step_700_long_pred))
            reduced_model5_long_pred.append(pca.transform(step_900_long_pred))


reduced_model1_long_pred = np.concatenate(reduced_model1_long_pred, axis=0)
reduced_model2_long_pred = np.concatenate(reduced_model2_long_pred, axis=0)
reduced_model3_long_pred = np.concatenate(reduced_model3_long_pred, axis=0)
reduced_model4_long_pred = np.concatenate(reduced_model4_long_pred, axis=0)
reduced_model5_long_pred = np.concatenate(reduced_model5_long_pred, axis=0)


In [ ]:
# Step 1: Instantiate PCA object
# To keep all components but see how many are needed for 95% variance, you can set n_components to 0.95
pca = PCA(n_components=5)

# Step 2: Fit PCA on the data
pca.fit(np.concatenate((train_sample, test_u), axis=0))

# Step 3: Transform the data to get the principal components
test_u_pca = pca.transform(test_u)

# The transformed data `test_u_pca` now has fewer dimensions, with 95% of the original variance retained
print("Original shape:", test_u.shape)
print("Reduced shape:", test_u_pca.shape)

# If you want to see the number of components selected
print("Number of components selected:", pca.n_components_)

Original shape: torch.Size([110000, 128])
Reduced shape: (110000, 5)
Number of components selected: 5


### Calculate KL divergence of principle components' distribution 

In [ ]:
kde_truth_pca = calculate_kde(pca.transform(np.concatenate([train_sample, test_u])))
x_range = np.linspace(np.concatenate([train_sample, test_u]).min()-5, np.concatenate([train_sample, test_u]).max()+5, 40000)
kde_step_100_pca = calculate_kde(reduced_model1_long_pred)
kde_step_300_pca = calculate_kde(reduced_model2_long_pred)
kde_step_500_pca = calculate_kde(reduced_model3_long_pred)
kde_step_700_pca = calculate_kde(reduced_model4_long_pred)
kde_step_900_pca = calculate_kde(reduced_model5_long_pred)


In [ ]:
kl_dim_tru_step_100 = []
kl_dim_tru_step_300 = []
kl_dim_tru_step_500 = []
kl_dim_tru_step_700 = []
kl_dim_tru_step_900 = []


for i in range(5):
      print(f"Dimension {i+1}")

      kl_dim_tru_step_100_i = kl_divergence(kde_truth_pca[i], kde_step_100_pca[i], x_range)
      kl_dim_tru_step_100.append(kl_dim_tru_step_100_i)
      print(f"KL divergence (PC distribution) True vs contract step 100: {kl_dim_tru_step_100_i}")

      kl_dim_tru_step_300_i = kl_divergence(kde_truth_pca[i], kde_step_300_pca[i], x_range)
      kl_dim_tru_step_300.append(kl_dim_tru_step_300_i)
      print(f"KL divergence (PC distribution) True vs contract step 300: {kl_dim_tru_step_300_i}")

      kl_dim_tru_step_500_i = kl_divergence(kde_truth_pca[i], kde_step_500_pca[i], x_range)
      kl_dim_tru_step_500.append(kl_dim_tru_step_500_i)
      print(f"KL divergence (PC distribution) True vs contract step 500: {kl_dim_tru_step_500_i}")

      kl_dim_tru_step_700_i = kl_divergence(kde_truth_pca[i], kde_step_700_pca[i], x_range)
      kl_dim_tru_step_700.append(kl_dim_tru_step_700_i)
      print(f"KL divergence (PC distribution) True vs contract step 700: {kl_dim_tru_step_700_i}")

      kl_dim_tru_step_900_i = kl_divergence(kde_truth_pca[i], kde_step_900_pca[i], x_range)
      kl_dim_tru_step_900.append(kl_dim_tru_step_900_i)
      print(f"KL divergence (PC distribution) True vs contract step 900: {kl_dim_tru_step_900_i}")


Dimension 1
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [2.47678009e-37 2.47678009e-37 2.47678009e-37 ... 2.47678009e-37
 2.47678009e-37 2.47678009e-37] q_x shape: 40000
KL divergence (PC distribution) True vs contract step 100: 186347.4720097805
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00118026 0.00118026 0.00118026 ... 0.00117907 0.00117907 0.00117907] q_x shape: 40000
KL divergence (PC distribution) True vs contract step 300: 9769.285633266796
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00066794 0.00066803 0.00066812 ... 0.00021954 0.00021935 0.00021916] q_x shape: 40000
KL divergence (PC distribution) True vs contract step 500: 4.281563488371166
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00104153 0.00104217 0.00104282 ... 0.00089628 0.00089505 0.0008938